## Pipeline Readiness Check and Metdata Calls


Local example settings.

In [1]:
%env INFER_REST_ENDPOINT=http://0.0.0.0:9000
%env INFER_GRPC_ENDPOINT=0.0.0.0:9000
%env SELDON_SCHEDULE_HOST=0.0.0.0:9004

env: INFER_REST_ENDPOINT=http://0.0.0.0:9000
env: INFER_GRPC_ENDPOINT=0.0.0.0:9000
env: SELDON_SCHEDULE_HOST=0.0.0.0:9004


Remote k8s cluster example settings - change as neeed for your needs.

In [2]:
#%env INFER_REST_ENDPOINT=http://172.19.255.1:80
#%env INFER_GRPC_ENDPOINT=172.19.255.1:80
#%env SELDON_SCHEDULE_HOST=172.19.255.2:9004

### Model Chain - Ready Check

We will check the readiness of the Pipeline after every change to model and pipeline.

In [3]:
!cat ./pipelines/tfsimples.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: tfsimples
spec:
  steps:
    - name: tfsimple1
    - name: tfsimple2
      inputs:
      - tfsimple1
      tensorMap:
        tfsimple1.outputs.OUTPUT0: INPUT0
        tfsimple1.outputs.OUTPUT1: INPUT1
  output:
    steps:
    - tfsimple2


In [4]:
!curl -Ik ${INFER_REST_ENDPOINT}/v2/pipelines/tfsimples/ready

In [5]:
!grpcurl -d '{"name":"tfsimples"}' \
    -plaintext \
    -import-path ../apis \
    -proto ../apis/mlops/v2_dataplane/v2_dataplane.proto \
    -rpc-header seldon-model:tfsimples.pipeline \
    ${INFER_GRPC_ENDPOINT} inference.GRPCInferenceService/ModelReady

ERROR:
  Code: Unimplemented
  Message: 


In [6]:
!seldon pipeline load -f ./pipelines/tfsimples.yaml
!seldon pipeline status tfsimples -w PipelineReady

{"pipelineName":"tfsimples", "versions":[{"pipeline":{"name":"tfsimples", "uid":"ciepit2i8ufs73flaitg", "version":1, "steps":[{"name":"tfsimple1"}, {"name":"tfsimple2", "inputs":["tfsimple1.outputs"], "tensorMap":{"tfsimple1.outputs.OUTPUT0":"INPUT0", "tfsimple1.outputs.OUTPUT1":"INPUT1"}}], "output":{"steps":["tfsimple2.outputs"]}, "kubernetesMeta":{}}, "state":{"pipelineVersion":1, "status":"PipelineReady", "reason":"created pipeline", "lastChangeTimestamp":"2023-06-29T14:47:16.365934922Z"}}]}


In [7]:
!seldon pipeline status tfsimples | jq .versions[0].state.modelsReady

null


In [8]:
!curl -Ik ${INFER_REST_ENDPOINT}/v2/pipelines/tfsimples/ready

In [9]:
!grpcurl -d '{"name":"tfsimples"}' \
    -plaintext \
    -import-path ../apis \
    -proto ../apis/mlops/v2_dataplane/v2_dataplane.proto \
    -rpc-header seldon-model:tfsimples.pipeline \
    ${INFER_GRPC_ENDPOINT} inference.GRPCInferenceService/ModelReady

{
  
}


In [10]:
!seldon model load -f ./models/tfsimple1.yaml 
!seldon model status tfsimple1 -w ModelAvailable 

{}
{}


In [11]:
!curl -Ik ${INFER_REST_ENDPOINT}/v2/pipelines/tfsimples/ready

In [12]:
!grpcurl -d '{"name":"tfsimples"}' \
    -plaintext \
    -import-path ../apis \
    -proto ../apis/mlops/v2_dataplane/v2_dataplane.proto \
    -rpc-header seldon-model:tfsimples.pipeline \
    ${INFER_GRPC_ENDPOINT} inference.GRPCInferenceService/ModelReady

{
  
}


In [13]:
!seldon model load -f ./models/tfsimple2.yaml 
!seldon model status tfsimple2 -w ModelAvailable | jq -M .

{}
{}


In [14]:
!curl -Ik ${INFER_REST_ENDPOINT}/v2/pipelines/tfsimples/ready

In [15]:
!grpcurl -d '{"name":"tfsimples"}' \
    -plaintext \
    -import-path ../apis \
    -proto ../apis/mlops/v2_dataplane/v2_dataplane.proto \
    -rpc-header seldon-model:tfsimples.pipeline \
    ${INFER_GRPC_ENDPOINT} inference.GRPCInferenceService/ModelReady

{
  "ready": true
}


In [16]:
!seldon pipeline status tfsimples | jq .versions[0].state.modelsReady

true


In [17]:
!seldon pipeline unload tfsimples

In [18]:
!curl -Ik ${INFER_REST_ENDPOINT}/v2/pipelines/tfsimples/ready

In [19]:
!grpcurl -d '{"name":"tfsimples"}' \
    -plaintext \
    -import-path ../apis \
    -proto ../apis/mlops/v2_dataplane/v2_dataplane.proto \
    -rpc-header seldon-model:tfsimples.pipeline \
    ${INFER_GRPC_ENDPOINT} inference.GRPCInferenceService/ModelReady

ERROR:
  Code: Unimplemented
  Message: 


Models will still be ready even though Pipeline terminated

In [20]:
!seldon pipeline status tfsimples | jq .versions[0].state.modelsReady

true


In [21]:
!seldon pipeline load -f ./pipelines/tfsimples.yaml
!seldon pipeline status tfsimples -w PipelineReady

{"pipelineName":"tfsimples", "versions":[{"pipeline":{"name":"tfsimples", "uid":"ciepj5qi8ufs73flaiu0", "version":1, "steps":[{"name":"tfsimple1"}, {"name":"tfsimple2", "inputs":["tfsimple1.outputs"], "tensorMap":{"tfsimple1.outputs.OUTPUT0":"INPUT0", "tfsimple1.outputs.OUTPUT1":"INPUT1"}}], "output":{"steps":["tfsimple2.outputs"]}, "kubernetesMeta":{}}, "state":{"pipelineVersion":1, "status":"PipelineReady", "reason":"created pipeline", "lastChangeTimestamp":"2023-06-29T14:47:51.626155116Z", "modelsReady":true}}]}


In [22]:
!curl -Ik ${INFER_REST_ENDPOINT}/v2/pipelines/tfsimples/ready

In [23]:
!grpcurl -d '{"name":"tfsimples"}' \
    -plaintext \
    -import-path ../apis \
    -proto ../apis/mlops/v2_dataplane/v2_dataplane.proto \
    -rpc-header seldon-model:tfsimples.pipeline \
    ${INFER_GRPC_ENDPOINT} inference.GRPCInferenceService/ModelReady

{
  "ready": true
}


In [24]:
!seldon pipeline status tfsimples | jq .versions[0].state.modelsReady

true


In [25]:
!seldon model unload tfsimple1
!seldon model unload tfsimple2

In [26]:
!seldon pipeline status tfsimples | jq .versions[0].state.modelsReady

null


In [27]:
!seldon pipeline unload tfsimples

### Kubernetes Resource Example

In [1]:
import os
os.environ["NAMESPACE"] = "seldon-mesh"

In [2]:
MESH_IP=!kubectl get svc seldon-mesh -n ${NAMESPACE} -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

'172.19.255.1'

In [3]:
!kubectl create -f ./pipelines/tfsimples.yaml -n ${NAMESPACE}

pipeline.mlops.seldon.io/tfsimples created


In [4]:
!kubectl wait --for condition=ready --timeout=1s pipeline --all -n ${NAMESPACE}

error: timed out waiting for the condition on pipelines/tfsimples


In [5]:
!kubectl get pipeline tfsimples -o jsonpath='{.status.conditions[0]}' -n ${NAMESPACE}

{"lastTransitionTime":"2022-11-14T10:25:31Z","status":"False","type":"ModelsReady"}

In [6]:
!kubectl create -f ./models/tfsimple1.yaml -n ${NAMESPACE}
!kubectl create -f ./models/tfsimple2.yaml -n ${NAMESPACE}

model.mlops.seldon.io/tfsimple1 created
model.mlops.seldon.io/tfsimple2 created


In [7]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n ${NAMESPACE}

pipeline.mlops.seldon.io/tfsimples condition met


In [8]:
!kubectl get pipeline tfsimples -o jsonpath='{.status.conditions[0]}' -n ${NAMESPACE}

{"lastTransitionTime":"2022-11-14T10:25:49Z","status":"True","type":"ModelsReady"}

In [57]:
!kubectl delete -f ./models/tfsimple1.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/tfsimple2.yaml -n ${NAMESPACE}
!kubectl delete -f ./pipelines/tfsimples.yaml -n ${NAMESPACE}

model.mlops.seldon.io "tfsimple1" deleted
model.mlops.seldon.io "tfsimple2" deleted
pipeline.mlops.seldon.io "tfsimples" deleted
